In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [6]:
with open("../data/client_dataset.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [7]:
df = pd.DataFrame(data)

In [8]:
def unique_work(s):
    works = re.split(r"(?=\d{4}-\d{2}-\d{2}\s-\s(?:\d{4}-\d{2}-\d{2}|:))", s)
    works_set = set(entry.strip() for entry in works if entry.strip())
    return works_set

df['unique_work'] = df['work_experience'].apply(unique_work)

In [9]:
# Текущая дата для открытых интервалов
current_date = datetime.now()

def calculate_experience_months(data):
    intervals = []

    # Извлечение временных интервалов
    for entry in data:
        match = re.match(r'(\d{4}-\d{2}-\d{2}) - (\d{4}-\d{2}-\d{2}|:)', entry)
        if match:
            start_date = datetime.strptime(match.group(1), '%Y-%m-%d')
            end_date = current_date if match.group(2) == ':' else datetime.strptime(match.group(2), '%Y-%m-%d')
            intervals.append((start_date, end_date))

    # Объединение пересекающихся интервалов
    intervals.sort()  # Сортируем по началу интервалов
    merged_intervals = []
    for start, end in intervals:
        if not merged_intervals or merged_intervals[-1][1] < start:
            merged_intervals.append((start, end))
        else:
            merged_intervals[-1] = (merged_intervals[-1][0], max(merged_intervals[-1][1], end))

    # Подсчёт суммарной продолжительности в месяцах
    total_months = sum((relativedelta(end, start).years * 12 + relativedelta(end, start).months) for start, end in merged_intervals)
    return total_months


df['work_experience_months'] = df['unique_work'].apply(calculate_experience_months)



In [10]:
df['count_works'] = df['unique_work'].apply(len)

In [11]:
df['avg_time_per_work'] = df['work_experience_months']/df['count_works']

In [12]:
df[['unique_work', 'count_works', 'work_experience_months', 'avg_time_per_work']].head(10)

,unique_work,count_works,work_experience_months,avg_time_per_work
0,"{2013-12-01 - : ""программист"" (по факту аналит...",1,131,131.000000
1,{2017-06-01 - 2018-05-01: Начальник отдела инф...,6,145,24.166667
2,{2017-06-01 - 2018-05-01: Начальник отдела инф...,6,145,24.166667
3,{2017-06-01 - 2018-05-01: Начальник отдела инф...,6,145,24.166667
4,{2017-06-01 - 2018-05-01: Начальник отдела инф...,6,145,24.166667
5,{2015-01-01 - 2015-07-01: Программист Разработ...,3,108,36.000000
6,{2015-01-01 - 2015-07-01: Программист Разработ...,4,111,27.750000
7,{2012-10-01 - 2016-12-01: Главный специалист п...,4,181,45.250000
8,{2012-10-01 - 2016-12-01: Главный специалист п...,4,181,45.250000
9,{2018-08-01 - 2019-05-01: Ведущий системный ар...,9,166,18.444444


In [16]:
print(df['unique_work'][1])

{'2017-06-01 - 2018-05-01: Начальник отдела информационных технологий - осуществление текущего и перспективного планирования работ в отделе; \r\n- определение полномочий, должностных обязанностей и ответственности сотрудников отдела, в соответствии с их должностными инструкциями; \r\n- определение потребности в подготовке подчинённого персонала, оценка результативности его обучения; \r\n- анализ деятельность отдела; \r\n- организация обеспечения бесперебойного функционирования технического оборудования и программных средств; \r\n- разработка и утверждение в установленном порядке должностных и рабочих инструкций, определяющих правила работы персонала в условиях функционирования автоматизированных систем управления и медицинской информационной системы; \r\n- проведение обучения правильной эксплуатации систем управления и вычислительной техники персонала учреждения, работающего с компьютерной техникой; \r\n- осуществление оперативной связи с отделениями и службами учреждения, организация 